In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
# key => 
# 산업안전보건법 + "1"
# 건설기술진흥법 + "2"
# 위험물안전관리법 + '3'
# 전기안전관리법 + "4"
# 화학물질관리법 + "5"

In [2]:
df_industrial = pd.read_excel("산업안전보건법.xlsx")
df_erection = pd.read_excel("건설기술진흥법.xlsx")
df_danger = pd.read_excel("위험물안전관리법.xlsx")
df_electric = pd.read_excel("전기안전관리법.xlsx")
df_chemistry = pd.read_excel("화학물질관리법.xlsx")
df_info = pd.DataFrame()
df_info = df_info.append(df_industrial,ignore_index = True)
df_info = df_info.append(df_erection,ignore_index = True)
df_info = df_info.append(df_danger,ignore_index = True)
df_info = df_info.append(df_electric,ignore_index = True)
df_info = df_info.append(df_chemistry,ignore_index = True)
df_info

,키,조내용
0,10011,제1조(목적) 이 법은 산업 안전 및 보건에 관한 기준을 확립하고 그 책임의 소재를...
1,20011,제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2020.5.2...
2,30011,"제3조(적용 범위) 이 법은 모든 사업에 적용한다. 다만, 유해ㆍ위험의 정도, 사업..."
3,40011,제4조(정부의 책무) 정부는 이 법의 목적을 달성하기 위하여 다음 각 호의 사항을 ...
4,50011,제5조(사업주 등의 의무) 사업주(제77조에 따른 특수형태근로종사자로부터 노무를 제...
...,...,...
452,600015,제60조(벌칙) 제43조제2항에 따라 즉시 신고를 하지 아니한 자는 2년 이하의 징...
453,610015,제61조(벌칙) 다음 각 호의 어느 하나에 해당하는 자는 1년 이하의 징역 또는 3...
454,620015,제62조(벌칙) 다음 각 호의 어느 하나에 해당하는 자는 6개월 이하의 징역 또는 ...
455,630015,"제63조(양벌규정) 법인의 대표자나 법인 또는 개인의 대리인, 사용인, 그 밖의 종..."


In [3]:
# 불용어 리스트
# 불용어, 불필요 단어 제거
stop_words_df = pd.read_excel("stopwords.xlsx")
stop_words_df

,불용어
0,아
1,휴
2,아이구
3,아이쿠
4,아이고
...,...
694,따른
695,필요한
696,등을
697,해당하는


In [4]:
posts = df_info.get("조내용")
posts_key = df_info.get("키")
key = [i for i in posts_key]
posts

0      제1조(목적) 이 법은 산업 안전 및 보건에 관한 기준을 확립하고 그 책임의 소재를...
1      제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2020.5.2...
2      제3조(적용 범위) 이 법은 모든 사업에 적용한다. 다만, 유해ㆍ위험의 정도, 사업...
3      제4조(정부의 책무) 정부는 이 법의 목적을 달성하기 위하여 다음 각 호의 사항을 ...
4      제5조(사업주 등의 의무) 사업주(제77조에 따른 특수형태근로종사자로부터 노무를 제...
                             ...                        
452    제60조(벌칙) 제43조제2항에 따라 즉시 신고를 하지 아니한 자는 2년 이하의 징...
453    제61조(벌칙) 다음 각 호의 어느 하나에 해당하는 자는 1년 이하의 징역 또는 3...
454    제62조(벌칙) 다음 각 호의 어느 하나에 해당하는 자는 6개월 이하의 징역 또는 ...
455    제63조(양벌규정) 법인의 대표자나 법인 또는 개인의 대리인, 사용인, 그 밖의 종...
456    제64조(과태료) 다음 각 호의 어느 하나에 해당하는 자에게는 1천만원 이하의 과태...
Name: 조내용, Length: 457, dtype: object

In [5]:
def get_key(k) :
    
    if k[-1] == "1" :
        s = "OSH"+k[0:(len(k)-4)]
    elif k[-1]=="2":
        s = "CTP"+k[0:(len(k)-4)]
    elif k[-1]=="3":
        s = "HSM"+k[0:(len(k)-4)]
    elif k[-1]=="4":
        s = "ESM"+k[0:(len(k)-4)]
    elif k[-1]=="5":
        s = "CMM"+k[0:(len(k)-4)]     
    if k[-3] != "0":
        s += "의 "+k[-3]   
  
    return s

def get_law(law):
    if law == 'OSH':
        num = '1'
    elif law == 'CTP':
        num = '2'
    elif law == 'HSM':
        num = '3'
    elif law == 'ESM':
        num = '4'
    elif law == 'CMM':
        num = '5'
    return num

In [6]:
def tokenizer_1(raw_texts, pos=["Noun","Alpha","Verb","Number"], stopword=list(stop_words_df.get("불용어"))):
    p = okt.pos(raw_texts, 
            norm=True,   # 정규화(normalization)
            stem=True    # 어간추출(stemming)
            )
    o = [word for word, tag in p if len(word) > 1 and tag in pos and word[0] not in stopword]
    return(o)

def tokenizer_2(raw_texts, pos=["Noun","Alpha","Verb","Number"], stop_words=list(stop_words_df.get("불용어"))):
    nouns = []
  
    for noun in tagger.nouns(raw_texts):
        if noun not in stop_words:
            nouns.append(noun)
    return nouns

In [7]:
from konlpy.tag import Okt
okt = Okt()

oo = okt.pos(posts[0],
        norm=True,   # 정규화(normalization)
        stem=True    # 어간추출(stemming)
        )
print(oo)

[('제', 'Noun'), ('1조', 'Number'), ('(', 'Punctuation'), ('목적', 'Noun'), (')', 'Punctuation'), ('이', 'Noun'), ('법', 'Noun'), ('은', 'Josa'), ('산업', 'Noun'), ('안전', 'Noun'), ('및', 'Noun'), ('보건', 'Noun'), ('에', 'Josa'), ('관', 'Noun'), ('한', 'Josa'), ('기준', 'Noun'), ('을', 'Josa'), ('확립', 'Noun'), ('하고', 'Josa'), ('그', 'Noun'), ('책임', 'Noun'), ('의', 'Josa'), ('소재', 'Noun'), ('를', 'Josa'), ('명확하다', 'Adjective'), ('하다', 'Verb'), ('산업', 'Noun'), ('재해', 'Noun'), ('를', 'Josa'), ('예방', 'Noun'), ('하고', 'Josa'), ('쾌적하다', 'Adjective'), ('작업', 'Noun'), ('환경', 'Noun'), ('을', 'Josa'), ('조', 'Modifier'), ('성함', 'Noun'), ('으로써', 'Josa'), ('노무', 'Noun'), ('를', 'Josa'), ('제공', 'Noun'), ('하다', 'Verb'), ('사람', 'Noun'), ('의', 'Josa'), ('안전', 'Noun'), ('및', 'Noun'), ('보건', 'Noun'), ('을', 'Josa'), ('유지', 'Noun'), ('ㆍ', 'Foreign'), ('증진', 'Noun'), ('함', 'Noun'), ('을', 'Josa'), ('목적', 'Noun'), ('으로', 'Josa'), ('하다', 'Verb'), ('.', 'Punctuation'), ('<', 'Punctuation'), ('개정', 'Noun'), ('2020.5', 'Number'), ('.', '

In [8]:
from konlpy.tag import Mecab

tagger = Mecab('C:/mecab/mecab-ko-dic')
# from eunjeon import Mecab
# tagger = Mecab()
# 윈도우에서 Mecab 쓰려면 Mecab 폴더를 C드라이브에 만든 후 라이브러리 설치한 다음에 경로지정을 해주어야 함

tokenizer_2(posts[80])

['기계', '대여자', '조치', '기계', '건축물', '대여', '필요', '안전', '조치', '보건', '조치']

In [9]:
token = []
for i in range(len(posts)):
    token.append(tokenizer_2(posts[i]))
df_info['token'] = token
df_info

,키,조내용,token
0,10011,제1조(목적) 이 법은 산업 안전 및 보건에 관한 기준을 확립하고 그 책임의 소재를...,"[목적, 법, 안전, 보건, 기준, 확립, 책임, 소재, 재해, 예방, 작업, 환경..."
1,20011,제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2020.5.2...,"[정의, 법, 사용, 용어, 뜻, 개정, 산업재, 노무, 제공, 사람, 업무, 관계..."
2,30011,"제3조(적용 범위) 이 법은 모든 사업에 적용한다. 다만, 유해ㆍ위험의 정도, 사업...","[적용, 범위, 법, 사업, 적용, 유해, 위험, 정도, 사업, 종류, 사업장, 상..."
3,40011,제4조(정부의 책무) 정부는 이 법의 목적을 달성하기 위하여 다음 각 호의 사항을 ...,"[정부, 책무, 정부, 법, 목적, 달성, 이행, 책무, 개정, 정부, 효율, 수행..."
4,50011,제5조(사업주 등의 의무) 사업주(제77조에 따른 특수형태근로종사자로부터 노무를 제...,"[사업주, 의무, 사업주, 특수, 형태, 근로, 종사자, 노무, 제공, 물건, 수거..."
...,...,...,...
452,600015,제60조(벌칙) 제43조제2항에 따라 즉시 신고를 하지 아니한 자는 2년 이하의 징...,"[벌칙, 조제, 신고, 이하, 징역, 억, 원, 이하, 벌금]"
453,610015,제61조(벌칙) 다음 각 호의 어느 하나에 해당하는 자는 1년 이하의 징역 또는 3...,"[벌칙, 이하, 징역, 천만, 원, 이하, 벌금, 개정, 조제, 허가, 조건, 이행..."
454,620015,제62조(벌칙) 다음 각 호의 어느 하나에 해당하는 자는 6개월 이하의 징역 또는 ...,"[벌칙, 개월, 이하, 징역, 만, 원, 이하, 벌금, 개정, 조제, 단서, 위반,..."
455,630015,"제63조(양벌규정) 법인의 대표자나 법인 또는 개인의 대리인, 사용인, 그 밖의 종...","[양벌규정, 법인, 대표자, 법인, 개인, 대리인, 사용인, 밖, 종업원, 법인, ..."


Gephi 파일 직접 생성하기

In [10]:
from collections import Counter

edge_list = []
for token_list in df_info['token']:
    num_token = len(token_list)
    
    if num_token > 0:
        for i in range(num_token-1):
            for j in range(i+1, num_token):
                edge_list += [tuple(sorted([token_list[i], token_list[j]]))]    # node 간 위해 sorted 사용
edges = list(Counter(edge_list).items())

edges

[(('목적', '법'), 35),
 (('목적', '안전'), 168),
 (('목적', '보건'), 32),
 (('기준', '목적'), 48),
 (('목적', '확립'), 3),
 (('목적', '책임'), 3),
 (('목적', '소재'), 2),
 (('목적', '재해'), 14),
 (('목적', '예방'), 31),
 (('목적', '작업'), 11),
 (('목적', '환경'), 22),
 (('목적', '조성'), 2),
 (('노무', '목적'), 4),
 (('목적', '제공'), 16),
 (('목적', '사람'), 9),
 (('목적', '유지'), 8),
 (('목적', '목적'), 14),
 (('개정', '목적'), 20),
 (('법', '안전'), 418),
 (('법', '보건'), 136),
 (('기준', '법'), 107),
 (('법', '확립'), 4),
 (('법', '책임'), 2),
 (('법', '소재'), 1),
 (('법', '재해'), 38),
 (('법', '예방'), 51),
 (('법', '작업'), 53),
 (('법', '환경'), 90),
 (('법', '조성'), 5),
 (('노무', '법'), 15),
 (('법', '제공'), 41),
 (('법', '사람'), 68),
 (('법', '유지'), 26),
 (('개정', '법'), 164),
 (('보건', '안전'), 2687),
 (('기준', '안전'), 871),
 (('안전', '확립'), 11),
 (('안전', '책임'), 24),
 (('소재', '안전'), 2),
 (('안전', '재해'), 216),
 (('안전', '예방'), 383),
 (('안전', '작업'), 468),
 (('안전', '환경'), 449),
 (('안전', '조성'), 20),
 (('노무', '안전'), 36),
 (('안전', '제공'), 393),
 (('사람', '안전'), 320),
 (('안전', '안전'), 3770),
 (('안

In [11]:
import networkx as nx

G = nx.Graph((x, y, {'weight':v}) for (x,y), v in edges)
G

In [12]:
""" Community 추출 """
import community
from community import community_louvain as cl

partition = cl.best_partition(G)
nx.set_node_attributes(G, partition, "community")   # graph G에 community 속성 추가


""" Gephi file 작성 """
nx.write_gexf(G, 'industrial.gexf')

## 코사인 유사도 csv 파일 생성

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

countv = CountVectorizer(stop_words = list(stop_words_df.get("불용어")))
sp_mat = countv.fit_transform(posts)
# 희소행렬을 np array로
sp_mat.toarray()

C:\Users\assay\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['가서', '같은', '것과', '결과에', '결론을', '관계가', '관련이', '그런', '그럼에도', '그렇게', '그에', '그치지', '김에', '까닭에', '낫다', '년도', '논하지', '누가', '다시', '달려', '대로', '대해', '되는', '되다', '되어', '들면', '들자면', '듯하다', '따르는', '따름이다', '따지지', '때가', '만은', '만이', '많은', '말하면', '말할것도', '몰라도', '몰랏다', '못하다', '못하다하기보다는', '미치다', '바꾸어서', '바꿔', '방면으로', '보면', '보아', '부류의', '비길수', '비추어', '뿐만', '사람들', '상대적으로', '생각이다', '서술한바와같이', '쓰여', '아니다', '아니라', '안다', '안된다', '않고', '않기', '않는다면', '않다', '않다면', '않도록', '않으면', '알겠는가', '어쩔수', '없고', '없다', '예를', '외에', '요만한', '우에', '위에서', '이렇게', '이로', '이르다', '이와', '이유는', '인하여', '임에', '점에서', '정도에', '정도의', '종합한것과같이', '주저하지', '줄은', '지경이다', '틀림없다', '편이', '하고', '하기', '하기만', '하는', '하는것만', '하는것이', '하면', '하지', '한하다', '할수록', '함으로써', '해도', '해서는', '형식으로', '힘이'] not in stop_words.
  warnings.warn('Your stop_words may be inconsis

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorize = CountVectorizer(
    tokenizer=tokenizer_2, # 문장에 대한 tokenizer (위에 정의한 함수 이용)
    min_df=1,            # 단어가 출현하는 최소 문서의 개수
)

X = vectorize.fit_transform(posts)
X.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [15]:
cosine_result = []
for x in range(len(df_info)):
    tmp = []
    for y in range(len(df_info)):
        tmp.append(cosine_similarity(X[x], X[y])[0][0])
    cosine_result.append(tmp)

In [16]:
# 모든 조에 해당하는 cosine similarity 결과 값을
# DataFrame 형태로 변환
cosine_result_df = pd.DataFrame(cosine_result)
cosine_result_df

,0,1,2,3,4,5,6,7,8,9,...,447,448,449,450,451,452,453,454,455,456
0,1.000000,0.203186,0.052870,0.567462,0.383571,0.221470,0.047023,0.063764,0.178279,0.061602,...,0.158610,0.041996,0.046676,0.049041,0.057183,0.000000,0.041345,0.018692,0.000000,0.017168
1,0.203186,1.000000,0.306160,0.136919,0.353960,0.237855,0.085990,0.092311,0.095499,0.200266,...,0.085940,0.068264,0.009484,0.043843,0.039836,0.011790,0.036403,0.030384,0.064073,0.049997
2,0.052870,0.306160,1.000000,0.075942,0.250958,0.180656,0.000000,0.085333,0.046271,0.105523,...,0.226415,0.179847,0.066630,0.056004,0.040814,0.082832,0.039346,0.040025,0.000000,0.040844
3,0.567462,0.136919,0.075942,1.000000,0.346898,0.262573,0.082553,0.178091,0.227625,0.062922,...,0.151884,0.026810,0.044697,0.046961,0.067796,0.037043,0.035192,0.017900,0.014380,0.018266
4,0.383571,0.353960,0.250958,0.346898,1.000000,0.573409,0.121747,0.185727,0.230792,0.202025,...,0.205329,0.018122,0.100707,0.093113,0.112803,0.100157,0.101099,0.064529,0.019439,0.039510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,0.000000,0.011790,0.082832,0.037043,0.100157,0.000000,0.036835,0.037462,0.000000,0.077209,...,0.000000,0.131590,0.585018,0.230497,0.217574,1.000000,0.388650,0.322139,0.000000,0.268964
453,0.041345,0.036403,0.039346,0.035192,0.101099,0.058864,0.113734,0.050419,0.060308,0.050429,...,0.009837,0.109388,0.225788,0.675187,0.541073,0.388650,1.000000,0.688597,0.000000,0.760186
454,0.018692,0.030384,0.040025,0.017900,0.064529,0.000000,0.083063,0.036204,0.000000,0.024872,...,0.013342,0.084781,0.188457,0.584115,0.507114,0.322139,0.688597,1.000000,0.045472,0.531415
455,0.000000,0.064073,0.000000,0.014380,0.019439,0.000000,0.000000,0.009695,0.009857,0.007493,...,0.032154,0.051081,0.000000,0.095440,0.099362,0.000000,0.000000,0.045472,1.000000,0.055684


In [17]:
key = []
for i in df_info['키']:
    i = str(i)
    key.append(get_key(i))
print(key)

['OSH1', 'OSH2', 'OSH3', 'OSH4', 'OSH5', 'OSH6', 'OSH7', 'OSH8', 'OSH9', 'OSH10', 'OSH11', 'OSH12', 'OSH13', 'OSH14', 'OSH15', 'OSH16', 'OSH17', 'OSH18', 'OSH19', 'OSH20', 'OSH21', 'OSH22', 'OSH23', 'OSH24', 'OSH25', 'OSH26', 'OSH27', 'OSH28', 'OSH29', 'OSH30', 'OSH31', 'OSH32', 'OSH33', 'OSH34', 'OSH35', 'OSH36', 'OSH37', 'OSH38', 'OSH39', 'OSH40', 'OSH41', 'OSH42', 'OSH43', 'OSH44', 'OSH45', 'OSH46', 'OSH47', 'OSH48', 'OSH49', 'OSH50', 'OSH51', 'OSH52', 'OSH53', 'OSH54', 'OSH55', 'OSH56', 'OSH57', 'OSH58', 'OSH59', 'OSH60', 'OSH61', 'OSH62', 'OSH63', 'OSH64', 'OSH65', 'OSH66', 'OSH67', 'OSH68', 'OSH69', 'OSH70', 'OSH71', 'OSH72', 'OSH73', 'OSH74', 'OSH75', 'OSH76', 'OSH77', 'OSH78', 'OSH79', 'OSH80', 'OSH81', 'OSH82', 'OSH83', 'OSH84', 'OSH85', 'OSH86', 'OSH87', 'OSH88', 'OSH89', 'OSH90', 'OSH91', 'OSH92', 'OSH93', 'OSH94', 'OSH95', 'OSH96', 'OSH97', 'OSH98', 'OSH99', 'OSH100', 'OSH101', 'OSH102', 'OSH103', 'OSH104', 'OSH105', 'OSH106', 'OSH107', 'OSH108', 'OSH109', 'OSH110', 'OSH111

In [18]:
cosine_result_df.columns = key
cosine_result_df.index = key
cosine_result_df

,OSH1,OSH2,OSH3,OSH4,OSH5,OSH6,OSH7,OSH8,OSH9,OSH10,...,CMM55,CMM56,CMM57,CMM58,CMM59,CMM60,CMM61,CMM62,CMM63,CMM64
OSH1,1.000000,0.203186,0.052870,0.567462,0.383571,0.221470,0.047023,0.063764,0.178279,0.061602,...,0.158610,0.041996,0.046676,0.049041,0.057183,0.000000,0.041345,0.018692,0.000000,0.017168
OSH2,0.203186,1.000000,0.306160,0.136919,0.353960,0.237855,0.085990,0.092311,0.095499,0.200266,...,0.085940,0.068264,0.009484,0.043843,0.039836,0.011790,0.036403,0.030384,0.064073,0.049997
OSH3,0.052870,0.306160,1.000000,0.075942,0.250958,0.180656,0.000000,0.085333,0.046271,0.105523,...,0.226415,0.179847,0.066630,0.056004,0.040814,0.082832,0.039346,0.040025,0.000000,0.040844
OSH4,0.567462,0.136919,0.075942,1.000000,0.346898,0.262573,0.082553,0.178091,0.227625,0.062922,...,0.151884,0.026810,0.044697,0.046961,0.067796,0.037043,0.035192,0.017900,0.014380,0.018266
OSH5,0.383571,0.353960,0.250958,0.346898,1.000000,0.573409,0.121747,0.185727,0.230792,0.202025,...,0.205329,0.018122,0.100707,0.093113,0.112803,0.100157,0.101099,0.064529,0.019439,0.039510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CMM60,0.000000,0.011790,0.082832,0.037043,0.100157,0.000000,0.036835,0.037462,0.000000,0.077209,...,0.000000,0.131590,0.585018,0.230497,0.217574,1.000000,0.388650,0.322139,0.000000,0.268964
CMM61,0.041345,0.036403,0.039346,0.035192,0.101099,0.058864,0.113734,0.050419,0.060308,0.050429,...,0.009837,0.109388,0.225788,0.675187,0.541073,0.388650,1.000000,0.688597,0.000000,0.760186
CMM62,0.018692,0.030384,0.040025,0.017900,0.064529,0.000000,0.083063,0.036204,0.000000,0.024872,...,0.013342,0.084781,0.188457,0.584115,0.507114,0.322139,0.688597,1.000000,0.045472,0.531415
CMM63,0.000000,0.064073,0.000000,0.014380,0.019439,0.000000,0.000000,0.009695,0.009857,0.007493,...,0.032154,0.051081,0.000000,0.095440,0.099362,0.000000,0.000000,0.045472,1.000000,0.055684


In [19]:
cosine_result_df.to_csv('industrial.csv')

In [28]:
cosine_result_df.to_csv('cosine.csv', index=True)

In [29]:
cosine_result_df

,OSH1,OSH2,OSH3,OSH4,OSH5,OSH6,OSH7,OSH8,OSH9,OSH10,...,CMM55,CMM56,CMM57,CMM58,CMM59,CMM60,CMM61,CMM62,CMM63,CMM64
OSH1,1.000000,0.203186,0.052870,0.567462,0.383571,0.221470,0.047023,0.063764,0.178279,0.061602,...,0.158610,0.041996,0.046676,0.049041,0.057183,0.000000,0.041345,0.018692,0.000000,0.017168
OSH2,0.203186,1.000000,0.306160,0.136919,0.353960,0.237855,0.085990,0.092311,0.095499,0.200266,...,0.085940,0.068264,0.009484,0.043843,0.039836,0.011790,0.036403,0.030384,0.064073,0.049997
OSH3,0.052870,0.306160,1.000000,0.075942,0.250958,0.180656,0.000000,0.085333,0.046271,0.105523,...,0.226415,0.179847,0.066630,0.056004,0.040814,0.082832,0.039346,0.040025,0.000000,0.040844
OSH4,0.567462,0.136919,0.075942,1.000000,0.346898,0.262573,0.082553,0.178091,0.227625,0.062922,...,0.151884,0.026810,0.044697,0.046961,0.067796,0.037043,0.035192,0.017900,0.014380,0.018266
OSH5,0.383571,0.353960,0.250958,0.346898,1.000000,0.573409,0.121747,0.185727,0.230792,0.202025,...,0.205329,0.018122,0.100707,0.093113,0.112803,0.100157,0.101099,0.064529,0.019439,0.039510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CMM60,0.000000,0.011790,0.082832,0.037043,0.100157,0.000000,0.036835,0.037462,0.000000,0.077209,...,0.000000,0.131590,0.585018,0.230497,0.217574,1.000000,0.388650,0.322139,0.000000,0.268964
CMM61,0.041345,0.036403,0.039346,0.035192,0.101099,0.058864,0.113734,0.050419,0.060308,0.050429,...,0.009837,0.109388,0.225788,0.675187,0.541073,0.388650,1.000000,0.688597,0.000000,0.760186
CMM62,0.018692,0.030384,0.040025,0.017900,0.064529,0.000000,0.083063,0.036204,0.000000,0.024872,...,0.013342,0.084781,0.188457,0.584115,0.507114,0.322139,0.688597,1.000000,0.045472,0.531415
CMM63,0.000000,0.064073,0.000000,0.014380,0.019439,0.000000,0.000000,0.009695,0.009857,0.007493,...,0.032154,0.051081,0.000000,0.095440,0.099362,0.000000,0.000000,0.045472,1.000000,0.055684


In [32]:
a = pd.read_csv('cosine.csv', index_col=0)
a

,OSH1,OSH2,OSH3,OSH4,OSH5,OSH6,OSH7,OSH8,OSH9,OSH10,...,CMM55,CMM56,CMM57,CMM58,CMM59,CMM60,CMM61,CMM62,CMM63,CMM64
OSH1,1.000000,0.203186,0.052870,0.567462,0.383571,0.221470,0.047023,0.063764,0.178279,0.061602,...,0.158610,0.041996,0.046676,0.049041,0.057183,0.000000,0.041345,0.018692,0.000000,0.017168
OSH2,0.203186,1.000000,0.306160,0.136919,0.353960,0.237855,0.085990,0.092311,0.095499,0.200266,...,0.085940,0.068264,0.009484,0.043843,0.039836,0.011790,0.036403,0.030384,0.064073,0.049997
OSH3,0.052870,0.306160,1.000000,0.075942,0.250958,0.180656,0.000000,0.085333,0.046271,0.105523,...,0.226415,0.179847,0.066630,0.056004,0.040814,0.082832,0.039346,0.040025,0.000000,0.040844
OSH4,0.567462,0.136919,0.075942,1.000000,0.346898,0.262573,0.082553,0.178091,0.227625,0.062922,...,0.151884,0.026810,0.044697,0.046961,0.067796,0.037043,0.035192,0.017900,0.014380,0.018266
OSH5,0.383571,0.353960,0.250958,0.346898,1.000000,0.573409,0.121747,0.185727,0.230792,0.202025,...,0.205329,0.018122,0.100707,0.093113,0.112803,0.100157,0.101099,0.064529,0.019439,0.039510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CMM60,0.000000,0.011790,0.082832,0.037043,0.100157,0.000000,0.036835,0.037462,0.000000,0.077209,...,0.000000,0.131590,0.585018,0.230497,0.217574,1.000000,0.388650,0.322139,0.000000,0.268964
CMM61,0.041345,0.036403,0.039346,0.035192,0.101099,0.058864,0.113734,0.050419,0.060308,0.050429,...,0.009837,0.109388,0.225788,0.675187,0.541073,0.388650,1.000000,0.688597,0.000000,0.760186
CMM62,0.018692,0.030384,0.040025,0.017900,0.064529,0.000000,0.083063,0.036204,0.000000,0.024872,...,0.013342,0.084781,0.188457,0.584115,0.507114,0.322139,0.688597,1.000000,0.045472,0.531415
CMM63,0.000000,0.064073,0.000000,0.014380,0.019439,0.000000,0.000000,0.009695,0.009857,0.007493,...,0.032154,0.051081,0.000000,0.095440,0.099362,0.000000,0.000000,0.045472,1.000000,0.055684
